# 生成 AT 指令文档（带 参数(JSON) 字段）
此 Notebook 从 CSV → YAML → RST → HTML，使用 CSV 中 `参数(JSON)` 字段作为参数来源。

In [ ]:
!pip install pandas jinja2 ruamel.yaml sphinx sphinx-rtd-theme
print("✅ 依赖安装完成")

In [ ]:
import os
import pandas as pd
import json
from ruamel.yaml import YAML
from jinja2 import Environment
from sphinx.application import Sphinx

In [ ]:
# 读取 CSV，假设 CSV 含有 `参数(JSON)` 字段，该字段是 JSON 字符串
CSV_INPUT = 'input.csv'
df = pd.read_csv(CSV_INPUT, encoding='utf-8')

# 将参数(JSON) 列解析为 Python 结构
def safe_parse_param_json(s):
    try:
        return json.loads(s) if isinstance(s, str) and s.strip() else []
    except Exception as e:
        print(f"⚠️ 解析参数(JSON) 失败: {e}, value={s}")
        return []

df['参数_parsed'] = df['参数(JSON)'].apply(safe_parse_param_json)
df.head()

In [ ]:
# 将 CSV 转为 YAML，每一条命令一文件或写一个集合 YAML
yaml = YAML()
yaml.default_flow_style = False

OUT_YAML_DIR = 'yaml_output'
os.makedirs(OUT_YAML_DIR, exist_ok=True)

for idx, row in df.iterrows():
    cmd = row['命令']
    obj = {
        '命令': cmd,
        '命令标题': row.get('命令标题', ''),
        '命令类型': row.get('命令类型', ''),
        '响应': row.get('响应', ''),
        '示例命令': row.get('示例命令', ''),
        '功能描述': row.get('功能描述', ''),
        '备注': row.get('备注', ''),
        '参数': row['参数_parparsed'] if '参数_parparsed' in row else safe_parse_param_json(row.get('参数(JSON)', ''))
    }
    fn = os.path.join(OUT_YAML_DIR, f"{cmd}.yaml")
    with open(fn, 'w', encoding='utf-8') as f:
        yaml.dump(obj, f)

print("✅ YAML 文件已生成，目录：", OUT_YAML_DIR)

In [ ]:
# 由 YAML → RST（简化版）
env = Environment()
TEMPLATE = '''
{{ command }} — {{ title }}\n
{{ '=' * (command|length + 3 + title|length) }}\n
\n
命令类型: {{ ctype }}\n
响应: |\n
{% for l in response.splitlines() %}  {{ l }}\n{% endfor %}
\n
参数:\n
{% if params %}
{% for p in params %}- **{{ p.name }}**：
{% if p.valmap %}
\n
{% for k, v in p.valmap.items() %}    - {{ k }}：{{ v }}\n{% endfor %}
{% else %}{{ p.desc }}\n{% endif %}
{% endfor %}
{% else %}无\n{% endif %}
\n
说明: {{ note }}\n
\n
示例命令: |\n
{% for l in example.splitlines() %}  {{ l }}\n{% endfor %}
'''

OUT_RST_DIR = 'rst_output'
os.makedirs(OUT_RST_DIR, exist_ok=True)

for idx, row in df.iterrows():
    cmd = row['命令']
    params = safe_parse_param_json(row.get('参数(JSON)', ''))
    rst = env.from_string(TEMPLATE).render(
        command=cmd, title=row.get('命令标题', ''), ctype=row.get('命令类型', ''),
        response=row.get('响应', ''), params=params,
        note=row.get('备注', ''), example=row.get('示例命令', '')
    )
    with open(os.path.join(OUT_RST_DIR, f"{cmd}.rst"), 'w', encoding='utf-8') as f:
        f.write(rst)

print("✅ RST 文件已生成，目录：", OUT_RST_DIR)

In [ ]:
# 最后，用 Sphinx 构建 HTML（假定已有 conf.py 在 docs/）
Sphinx(srcdir='rst_output', confdir='docs', outdir='docs/build/html', buildername='html').build()
print("✅ HTML 构建完成，输出目录：docs/build/html")